### Webscraping script

Note that if you do not see the opened website move at all within about 5 seconds upon running the codes, manually scroll down the website (should not have any issue with this now)

You can use this script for any shopee website (that is of the same format as the examples given below). Just change the url and the number of iterations within the outermost for loop. Number of for loops = `ceiling of (number of reviews/6)`. If there are 169 reviews, number of iterations will be 169/6 = ceiling(28.16) = 29. 

How to install chromedriver

https://www.youtube.com/watch?v=dz59GsdvUF8

(Nadya) textrank/ noun chunks

(Jing Han)data cleaning: to spot spelling errors using pre-existing libraries(?), remove punctuations, remove stop words, make lower case

do both (bigrams and trigrams) and (sentences) to get top sentences

summarisation - textrank and can find some other methods

use noun chunks/ (bigrams & trigrams) - spaCy 

topic modelling - LDA

lemmatization, word-forms (refer to telegram chat)

roberta - spelling issues not an issue

In [1]:
# Over-iterate the number of pages and drop duplicates
# Get upper bound of num of reviews - if 1.1k reviews get 1.2k /6 = 200 iterations

from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

num_reviews = 0

webdriver_path = "C:/webdriver/chromedriver"
url = "https://shopee.sg/Iphone-case-Iphone-11-pro-max-case-iphone-12-pro-max-Iphone-7-plus-case-TPU-silicone-mobile-phone-case-iphone-12-mini-Rubik's-cube-style-Silicone-Phone-Case-For-iPhone-11-Pro-Max-X-Xr-Xs-Max-7-8-Plus-Se-2020-12-pro-max-12-mini-i.303218718.5264408541"
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')

browser = webdriver.Chrome(webdriver_path, options=options)
browser.get(url)

source = browser.page_source
soup = BeautifulSoup(source,'lxml')
for i in soup.find_all('div', class_ = 'OitLRu'):
    # number of reviews will be the second item 
    num_reviews = i.text
# print(num_reviews)

#checking if the reviews will be in the form of "368" or "1.2k" for example
if 'k' in num_reviews:
    num_reviews = num_reviews[:-1]
    num_reviews = float(num_reviews) + 0.1
    num_reviews = int(num_reviews * 1000)
else:
    num_reviews = int(num_reviews) + 100
print(num_reviews)
# num_iterations is needed to determine the end point of the webscraping process
num_iterations = int(num_reviews/6) + 1
print(num_iterations)

2100
351


In [2]:
from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

webdriver_path = "C:/webdriver/chromedriver"
# url = "https://shopee.sg/3Ply-Medical-Adult-Kids-Mask-50pcs-Ear-loop-face-mask-CE-certified-Local-ready-stock-(30pcs-box)-wholesale-available-i.265826573.3961492512"
url = "https://shopee.sg/Iphone-case-Iphone-11-pro-max-case-iphone-12-pro-max-Iphone-7-plus-case-TPU-silicone-mobile-phone-case-iphone-12-mini-Rubik's-cube-style-Silicone-Phone-Case-For-iPhone-11-Pro-Max-X-Xr-Xs-Max-7-8-Plus-Se-2020-12-pro-max-12-mini-i.303218718.5264408541"

options = webdriver.ChromeOptions()
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')

browser = webdriver.Chrome(webdriver_path, options=options)
browser.get(url)

comment_list = []
star_rating = []

try:
    browser.implicitly_wait(30)
    try:
        # number in the for loop is (ceiling(number of reviews/6)). If number of reviews is 169, take 169/6 = 29 and 
        # put inside for loop
        
        for i in range(num_iterations):
            # scroll to the bottom of the page as shopee uses lazy loading
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            for e in browser.find_elements_by_class_name('shopee-product-rating__content'):
                comment_list.append(e.text)
                time.sleep(0.5)
                
            # star ratings
            for rating in browser.find_elements_by_css_selector('.shopee-product-rating'):
                stars = rating.find_elements_by_css_selector('.icon-rating-solid')
                star_rating.append(len(stars))
                time.sleep(0.5)
            
            for e in browser.find_elements_by_xpath('//button[@class="shopee-icon-button shopee-icon-button--right " and not(@aria-disabled)]'):
                browser.implicitly_wait(5)
                time.sleep(0.5)
                e.click()
                browser.implicitly_wait(5)
        browser.quit()
        print("Webscrape successful!")
    except:
        browser.quit()
        print("Error!")
except NoSuchElementException: 
    browser.quit()

Webscrape successful!


In [3]:
print(len(comment_list))
print(len(star_rating))

2026
2026


In [4]:
df = pd.DataFrame(list(zip(comment_list, star_rating)), columns = ['Reviews', 'Ratings'])
df

,Reviews,Ratings
0,"Fits my iPhone 7 perfectly, very nice case! Re...",5
1,quality super good for the price! picture and ...,5
2,Very nice and cute phone case. Like the colour...,5
3,Items are well packed and looks really good. L...,5
4,love the colour! will purchase other colours i...,5
...,...,...
2021,,5
2022,,5
2023,,5
2024,,5


Obtaining dataframe of reviews that are not empty string

In [5]:
# check_length() will check if the sentence is of length 0 aka empty string
def check_length(sentence):
    return len(sentence)
checker = lambda x: check_length(x)

df2 = df.copy()
df2['length'] = df['Reviews'].apply(checker)
df2
# remove empty string
df2 = df2[df2['length'] != 0]
df2 = df2[df2['Reviews'] != '******The review has been hidden due to inappropriate content.******']
df2

,Reviews,Ratings,length
0,"Fits my iPhone 7 perfectly, very nice case! Re...",5,69
1,quality super good for the price! picture and ...,5,124
2,Very nice and cute phone case. Like the colour...,5,109
3,Items are well packed and looks really good. L...,5,108
4,love the colour! will purchase other colours i...,5,102
...,...,...,...
1040,casing is so pretty ! thankyou!!,5,32
1041,it’s cute! both look exact to picture,5,37
1043,Pretty ok case for the price.,5,30
1044,Used normal delivery and received within a wee...,5,96


Removing duplicate rows from the dataframe

In [6]:
df2 = df2.drop_duplicates()
df2

,Reviews,Ratings,length
0,"Fits my iPhone 7 perfectly, very nice case! Re...",5,69
1,quality super good for the price! picture and ...,5,124
2,Very nice and cute phone case. Like the colour...,5,109
3,Items are well packed and looks really good. L...,5,108
4,love the colour! will purchase other colours i...,5,102
...,...,...,...
1040,casing is so pretty ! thankyou!!,5,32
1041,it’s cute! both look exact to picture,5,37
1043,Pretty ok case for the price.,5,30
1044,Used normal delivery and received within a wee...,5,96


In [8]:
df2.to_csv("shopee_reviews.csv")